# Data Profiler and Schema Validation

Profiles given input data based on the custom queries you provide, and validates its schema against schema repository. 
You can find how to insert a schema to schema-repository in README.md

In [1]:
%%help

Magic,Example,Explanation
info,%%info,Outputs session information for the current Livy endpoint.
cleanup,%%cleanup -f,"Deletes all sessions for the current Livy endpoint, including this notebook's session. The force flag is mandatory."
delete,%%delete -f -s 0,Deletes a session by number for the current Livy endpoint. Cannot delete this kernel's session.
logs,%%logs,Outputs the current session's Livy logs.
configure,"%%configure -f{""executorMemory"": ""1000M"", ""executorCores"": 4}",Configure the session creation parameters. The force flag is mandatory if a session has already been created and the session will be dropped and recreated.Look at Livy's POST /sessions Request Body for a list of valid parameters. Parameters must be passed in as a JSON string.
spark,%%spark -o dfdf = spark.read.parquet('...,"Executes spark commands. Parameters: -o VAR_NAME: The Spark dataframe of name VAR_NAME will be available in the %%local Python context as a Pandas dataframe with the same name. -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe that will be pulled from Livy to Jupyter. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
sql,%%sql -o tables -qSHOW TABLES,"Executes a SQL query against the variable sqlContext (Spark v1.x) or spark (Spark v2.x). Parameters: -o VAR_NAME: The result of the SQL query will be available in the %%local Python context as a Pandas dataframe. -q: The magic will return None instead of the dataframe (no visualization). -m, -n, -r are the same as the %%spark parameters above."
local,%%locala = 1,All the code in subsequent lines will be executed locally. Code must be valid Python code.


## Spark job configuration parameters like memory and cores may vary from one job to other

In [2]:
%%configure -f
{"name":"data-profiler", 
 "executorMemory": "2GB", 
 "executorCores": 4, 
 "conf": {"spark.jars.packages": "com.databricks:spark-avro_2.11:4.0.0,com.github.gphat:censorinus_2.11:2.1.13"} 
}

## Set parameters that will be overwritten by values passed externally

In [3]:
val dataFormat = "data-format"
val delimiter = ""
val inputDataLocation = "input-data-location"
val appName = "app-name" 
val schemaRepoUrl = "schema-repo-url"
val scheRepoSubjectName = "subject-name"
val schemaVersionId = "schema-version"
val customQ1 = "custom-query-1"
val customQ1ResultThreshold = 0
val customQ1Operator = "custom-operator-1"
val customQ2 = "custom-query-2"
val customQ2ResultThreshold = 0
val customQ2Operator = "custom-operator-2"
val customQ3 = "custom-query-3"
val customQ3ResultThreshold = 0
val customQ3Operator = "custom-query-3"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
278,application_1550710474644_0438,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


dataFormat: String = data-format
delimiter: String = ""
inputDataLocation: String = input-data-location
appName: String = app-name
schemaRepoUrl: String = schema-repo-url
scheRepoSubjectName: String = subject-name
schemaVersionId: String = schema-version
customQ1: String = custom-query-1
customQ1ResultThreshold: Int = 0
customQ1Operator: String = custom-operator-1
customQ2: String = custom-query-2
customQ2ResultThreshold: Int = 0
customQ2Operator: String = custom-operator-2
customQ3: String = custom-query-3
customQ3ResultThreshold: Int = 0
customQ3Operator: String = custom-query-3


In [4]:
// Parameters
val dataFormat = "json"
val inputDataLocation = "s3a://bucket/prefix/generated.json"
val appName = "cust-profile-data-validation"
val schemaRepoUrl = "http://schemarepohostaddress"
val scheRepoSubjectName = "cust-profile"
val schemaVersionId = "0"
val customQ1 = "select CAST(count(_id) - count(distinct _id) as Long) as diff from dataset"
val customQ1ResultThreshold = 0
val customQ1Operator = "="
val customQ2 = "select CAST(length(phone) as Long) from dataset"
val customQ2ResultThreshold = 17
val customQ2Operator = "="
val customQ3 = "select CAST(count(distinct gender) as Long) from dataset"
val customQ3ResultThreshold = 3
val customQ3Operator = "<="


dataFormat: String = json
inputDataLocation: String = s3a://bucket/prefix/generated.json
appName: String = cust-profile-data-validation
schemaRepoUrl: String = http://schemarepohostaddress
scheRepoSubjectName: String = cust-profile
schemaVersionId: String = 0
customQ1: String = select CAST(count(_id) - count(distinct _id) as Long) as diff from dataset
customQ1ResultThreshold: Int = 0
customQ1Operator: String = =
customQ2: String = select CAST(length(phone) as Long) from dataset
customQ2ResultThreshold: Int = 17
customQ2Operator: String = =
customQ3: String = select CAST(count(distinct gender) as Long) from dataset
customQ3ResultThreshold: Int = 3
customQ3Operator: String = <=


## Setup datadog statsd interface

In [5]:
import github.gphat.censorinus.DogStatsDClient

val statsd = new DogStatsDClient(hostname = "localhost", port = 8125, prefix = "mlp.validator")

import github.gphat.censorinus.DogStatsDClient
statsd: github.gphat.censorinus.DogStatsDClient = github.gphat.censorinus.DogStatsDClient@34f583f5


## Read data, if data being read is CSV, it needs to have a header

In [6]:
val df = dataFormat match {
    case "parquet" => spark.read.parquet(inputDataLocation)
    case "json" => spark.read.json(inputDataLocation)
    case "csv" => spark.read.option("mode", "DROPMALFORMED").option("header", "true").option("delimiter", delimiter).csv(inputDataLocation)
    case _ => throw new Exception(s"$dataFormat, as a dataformat is not supported ")
}

df: org.apache.spark.sql.DataFrame = [_id: string, about: string ... 20 more fields]


### Publish some basic stats about the data. This can be extended further

In [7]:
val recordCount = df.count()
val numColumns = df.columns.size
statsd.histogram(name = "recordCount", value = recordCount, tags = Seq(s"appName:$appName", "data-validation", "env:dev"));
statsd.histogram(name = "numColumns", value = numColumns, tags = Seq(s"appName:$appName", "data-validation","env:dev"));

recordCount: Long = 7
numColumns: Int = 22


## Read registered schema from schema repository

### Utility method to call rest endpoint for schema

In [8]:
import java.io.IOException;

import org.apache.http.HttpEntity;
import org.apache.http.HttpResponse;
import org.apache.http.client.ClientProtocolException;
import org.apache.http.client.ResponseHandler;
import org.apache.http.client.methods.HttpGet;
import org.apache.http.impl.client.CloseableHttpClient;
import org.apache.http.impl.client.HttpClients;
import org.apache.http.util.EntityUtils;

def getSchema(url: String) : String = {
    val httpclient: CloseableHttpClient = HttpClients.createDefault()
    try {
      val httpget: HttpGet = new HttpGet(url)
      println("Executing request " + httpget.getRequestLine)
      val responseHandler: ResponseHandler[String] =
        new ResponseHandler[String]() {
          override def handleResponse(response: HttpResponse): String = {
            var status: Int = response.getStatusLine.getStatusCode
            if (status >= 200 && status < 300) {
              var entity: HttpEntity = response.getEntity
              if (entity != null) EntityUtils.toString(entity) else null
            } else {
              throw new ClientProtocolException(
                "Unexpected response status: " + status);
            }
          }
        }
       httpclient.execute(httpget, responseHandler)  
    } finally {
        httpclient.close()
        None
    }
}

import java.io.IOException
import org.apache.http.HttpEntity
import org.apache.http.HttpResponse
import org.apache.http.client.ClientProtocolException
import org.apache.http.client.ResponseHandler
import org.apache.http.client.methods.HttpGet
import org.apache.http.impl.client.CloseableHttpClient
import org.apache.http.impl.client.HttpClients
import org.apache.http.util.EntityUtils
getSchema: (url: String)String


#### Create url from input parameters and feth schema for specified version

In [9]:
val schema_url = s"$schemaRepoUrl/schema-repo/$scheRepoSubjectName/id/$schemaVersionId"
val publishedSchema = getSchema(schema_url) 

schema_url: String = http://schemarepohostaddress/schema-repo/cust-profile/id/0
Executing request GET http://schemarepohostaddress/schema-repo/cust-profile/id/0 HTTP/1.1
publishedSchema: String =
{
  "type" : "record",
  "name" : "MyClass",
  "namespace" : "com.test.avro",
  "fields" : [ {
    "name" : "_id",
    "type" : "string"
  }, {
    "name" : "index",
    "type" : "long"
  }, {
    "name" : "guid",
    "type" : "string"
  }, {
    "name" : "isActive",
    "type" : "boolean"
  }, {
    "name" : "balance",
    "type" : "string"
  }, {
    "name" : "picture",
    "type" : "string"
  }, {
    "name" : "age",
    "type" : "long"
  }, {
    "name" : "eyeColor",
    "type" : "string"
  }, {
    "name" : "name",
    "type" : "string"
  }, {
    "name" : "gender",
    "type" : "string"
  }, {
    "name" : "company",
    "type" : "string"
  }, {
    "name" : "email",
    "type" : "string"
  }, {
    "name" : "phone",
    "type" : "string"
  }, {
    "name...

### Convert Avro schema registered to Spark SQL Schema.

In [10]:
import com.databricks.spark.avro._
import org.apache.avro.Schema.Parser
val schema = new Parser().parse(publishedSchema)

import com.databricks.spark.avro.SchemaConverters
val structSchema =  SchemaConverters.toSqlType(schema).dataType

import com.databricks.spark.avro._
import org.apache.avro.Schema.Parser
schema: org.apache.avro.Schema = {"type":"record","name":"MyClass","namespace":"com.test.avro","fields":[{"name":"_id","type":"string"},{"name":"index","type":"long"},{"name":"guid","type":"string"},{"name":"isActive","type":"boolean"},{"name":"balance","type":"string"},{"name":"picture","type":"string"},{"name":"age","type":"long"},{"name":"eyeColor","type":"string"},{"name":"name","type":"string"},{"name":"gender","type":"string"},{"name":"company","type":"string"},{"name":"email","type":"string"},{"name":"phone","type":"string"},{"name":"address","type":"string"},{"name":"about","type":"string"},{"name":"registered","type":"string"},{"name":"latitude","type":"double"},{"name":"longitude","type":"double"},{"name":"tags","type":{"type":"array","items":"string"}},{"name":"friends","type...import com.databricks.spark.avro.SchemaConverters
structSchema: org.apache.spark.sql.types.DataType = StructType(StructField(_id

### Utility method to traverse schema tree and find the leaf node names

In [11]:
import scala.collection.mutable.ListBuffer
import org.apache.spark.sql.types._

def findFields(path: String, dt: DataType, columnNames: ListBuffer[String]): Unit = dt match {
    case s: StructType =>
      s.fields.foreach(f => findFields(path + "." + f.name, f.dataType, columnNames))
    case s: ArrayType => findFields(path, s.elementType, columnNames)
    case other =>
      columnNames += path.substring(1)
}

import scala.collection.mutable.ListBuffer
import org.apache.spark.sql.types._
findFields: (path: String, dt: org.apache.spark.sql.types.DataType, columnNames: scala.collection.mutable.ListBuffer[String])Unit


In [12]:
var dfColumnNames = new ListBuffer[String]()
findFields("", df.schema, dfColumnNames)

print(dfColumnNames.toList)

dfColumnNames: scala.collection.mutable.ListBuffer[String] = ListBuffer()
List(_id, about, address, age, balance, company, email, eyeColor, favoriteFruit, friends.id, friends.name, gender, greeting, guid, index, isActive, latitude, longitude, name, phone, picture, registered, tags)

In [13]:
var publishedSchemaDataColumnNames = new ListBuffer[String]()
findFields("", structSchema, publishedSchemaDataColumnNames)

print(publishedSchemaDataColumnNames.toList)

publishedSchemaDataColumnNames: scala.collection.mutable.ListBuffer[String] = ListBuffer()
List(_id, index, guid, isActive, balance, picture, age, eyeColor, name, gender, company, email, phone, address, about, registered, latitude, longitude, tags, friends.id, friends.name, greeting, favoriteFruit)

In [14]:
val sourceColumns = dfColumnNames.toSet
val publishedColumns = publishedSchemaDataColumnNames.toSet
val differenceColumns = publishedColumns.diff(sourceColumns)
val numDiffColumns = differenceColumns.size
print(s"Number of columns not matching the schema are: $numDiffColumns")
statsd.histogram(name = "numDiffColumns", value = numDiffColumns, tags = Seq(s"appName:$appName", "data-validation", "env:dev"));

sourceColumns: scala.collection.immutable.Set[String] = Set(friends.id, registered, name, latitude, email, guid, _id, tags, balance, age, longitude, company, favoriteFruit, friends.name, isActive, greeting, address, picture, about, eyeColor, phone, index, gender)
publishedColumns: scala.collection.immutable.Set[String] = Set(friends.id, registered, name, latitude, email, guid, _id, tags, balance, age, longitude, company, favoriteFruit, friends.name, isActive, greeting, address, picture, about, eyeColor, phone, index, gender)
differenceColumns: scala.collection.immutable.Set[String] = Set()
numDiffColumns: Int = 0
Number of columns not matching the schema are: 0

### Custom data quality checks

#### Utility function to assert results

In [15]:
def customCheck(val1 : Long, operator : String, threshold : Long) : Unit = {
    operator match {
        case ">" => try { assert(val1 > threshold) } catch { case e: AssertionError => print(e);System.exit(1)}
        case ">=" => try { assert(val1 >= threshold) } catch { case e: AssertionError => print(e);System.exit(1)}
        case "=" => try { assert(val1 == threshold) } catch { case e: AssertionError => print(e);System.exit(1)}
        case "<" => try { assert(val1 < threshold) } catch { case e: AssertionError => print(e);System.exit(1)}
        case "<=" => try { assert(val1 <= threshold) } catch { case e: AssertionError => print(e);System.exit(1)}
    }
}

customCheck: (val1: Long, operator: String, threshold: Long)Unit


#### Create a temporary table, make sure that sql statements return a Long value, to be sure cast results to Long in the queries

In [16]:
df.createOrReplaceTempView("dataset")

val res1 = spark.sql(customQ1).collect().toList(0).getAs[Long](0)
customCheck(res1, customQ1Operator, customQ1ResultThreshold)

val res2 = spark.sql(customQ2).collect().toList(0).getAs[Long](0)
customCheck(res2, customQ2Operator, customQ2ResultThreshold)

val res3 = spark.sql(customQ3).collect().toList(0).getAs[Long](0)
customCheck(res3, customQ3Operator, customQ3ResultThreshold)

res1: Long = 0
res2: Long = 17
res3: Long = 1
